In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
np.seterr(divide='ignore', invalid='ignore')

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 1

# Naive Bayes classifier model

In [2]:
import pickle

with open('../../datasets/processed/trec2007-1607061515-features.dat', 'rb') as infile:
    X = pickle.load(infile)

with open('../../datasets/processed/trec2007-1607061515-labels.dat', 'rb') as infile:
    Y = pickle.load(infile)

N, D = X.shape    

print('X is a %s \t %s' % (X.shape, type(X)))
print('Y is a %s \t %s' % (Y.shape, type(Y)))


X is a (75419, 10000) 	 <class 'numpy.ndarray'>
Y is a (75419, 1) 	 <class 'numpy.matrixlib.defmatrix.matrix'>


In [3]:
add_bias = lambda x: np.insert(x, 0, values=1, axis=1) # add bias term
convert_labels = lambda y: y*2 - 1

## split into test and training sets

In [4]:
permutated_indices = np.random.permutation(N)
X = X[permutated_indices]
Y = Y[permutated_indices]

N_train = int(np.round(N * 0.5))
X_train = X[:N_train]
Y_train = Y[:N_train]
X_test = X[N_train:]
Y_test = Y[N_train:]

#X_train, X_test = map(add_bias, [X_train, X_test])
Y_train, Y_test = map(convert_labels, [Y_train, Y_test])

print(X_train.shape)
print(Y_train.shape)
print()
print(X_test.shape)
print(Y_test.shape)

(37710, 10000)
(37710, 1)

(37709, 10000)
(37709, 1)


## training

In [5]:
%aimport naivebayes

parameters = naivebayes.train_naivebayes(features=X_train, labels=Y_train, ham_label=-1, spam_label=1)

In [6]:
%aimport naivebayes

O_train = naivebayes.test_naivebayes(parameters=parameters, features=X_train, ham_label=-1, spam_label=1)
print(O_train.shape)

(37710,)


## testing

In [7]:
%aimport naivebayes

O_test = naivebayes.test_naivebayes(parameters=parameters, features=X_test, ham_label=-1, spam_label=1)
print(O_test.shape)

(37709,)


## performance

In [11]:
%aimport performance

print('error training set:\t%.3f' % performance.get_error(Y_train, O_train))
print('error testing  set:\t%.3f' % performance.get_error(Y_test,  O_test))
print('false positive rate:\t%.3f' % performance.get_FPR(Y_test, O_test, ham_label=-1, spam_label=1))
print('false negative rate:\t%.3f' % performance.get_FNR(Y_test, O_test, ham_label=-1, spam_label=1))

error training set:	0.041
error testing  set:	0.042
false positive rate:	0.028
false negative rate:	0.049


## Compare to sklearn's implementation

In [9]:
from sklearn.naive_bayes import BernoulliNB

nb = BernoulliNB()
nb.fit(X_train, np.array(Y_train).ravel())

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [10]:
get_sk_error = lambda X, Y: 1 - nb.score(X, Y)
print('Train error Scikit-Learn NB:\t%.3f' % get_sk_error(X_train, Y_train))
print(' Test error Scikit-Learn NB:\t%.3f' % get_sk_error(X_test, Y_test))

Train error Scikit-Learn NB:	0.047
 Test error Scikit-Learn NB:	0.048
